# Goal: Transfer annotations from cortex-level bounding box to WSI-level
### Author: Fan Fan
### Date: August 22, 2024

#### Import packages

In [1]:
import os
import cv2
import argparse
import numpy as np
import json
from skimage import color
import openslide
import pandas as pd
import glob
import csv
import shutil

In [10]:
input_folder = '/Volumes/data/UMich/spatial_transcriptomic/qupath/annotation/'
output_folder = '/Volumes/data/UMich/spatial_transcriptomic/qupath/annotation/'
resize = 0.25
classname = 'TE+TL'

#### Get WSI id - change the string as needed

In [6]:
# Get the list of PNG files from the specified input folder
files = glob.glob(f'{input_folder}/*.png')

# Extract unique WSI (Whole Slide Image) IDs from filenames
wsi_ids = list({os.path.basename(file).split('_PAS')[0] for file in files})

# Output the list of unique WSI IDs
wsi_ids

['Xenium_V1_0021981', 'Xenium_V1_0021980']

#### Generate annotation-json file *without* holes

In [13]:
# Iterate through each unique WSI ID
for wsi_id in wsi_ids:
    print(f'Processing {wsi_id}')
    geoj_features = []  # Initialize list to store GeoJSON features
    
    # Get all segmentation files corresponding to the current WSI ID
    seg_files = glob.glob(f'{input_folder}/{wsi_id}*.png')
    
    # Process each segmentation file
    for seg_file in seg_files:
        # Extract x and y coordinates from the filename
        x_roi, y_roi = float(seg_file.split('_')[6]), float(seg_file.split('_')[7])
        
        # Load the binary contour from the PNG file
        binary_contour = cv2.imread(seg_file, cv2.IMREAD_GRAYSCALE)
        
        # Apply binary thresholding to create a binary mask
        _, binary_mask = cv2.threshold(binary_contour, 127, 255, cv2.THRESH_BINARY)
        
        # Find contours in the binary mask
        contours, _ = cv2.findContours(binary_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # Process each contour
        for contour in contours:
            # Convert contour coordinates from numpy array to list
            contour_coords = np.squeeze(contour).tolist()
            contour_coords.append(contour_coords[0])  # Ensure the polygon is closed
            
            # Scale and translate the coordinates to the original ROI
            contour_coords = [[(1 / resize) * (c[0]) + x_roi, (1 / resize) * (c[1]) + y_roi] for c in contour_coords]
            
            # Ensure the polygon is closed by appending the first point again
            contour_coords.append(contour_coords[0])
            
            # Create a GeoJSON feature for the current contour
            geojson_feature = {
                "type": "Feature",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [contour_coords]
                },
                "properties": {
                    "objectType": "annotation",
                    "classification": {
                        "name": classname,
                        "color": [96, 12, 26]
                    }
                }
            }
            
            # Append the GeoJSON feature to the list of features
            geoj_features.append(geojson_feature)
    
    # Create a GeoJSON object containing all the features
    geojson_obj = {
        "type": "FeatureCollection",
        "features": geoj_features
    }
    
    # Save the GeoJSON object to a file
    output_file = f'{output_folder}/{wsi_id}.geojson'
    with open(output_file, 'w') as f:
        json.dump(geojson_obj, f)

    print(f'Saved GeoJSON for {wsi_id} to {output_file}')


Processing Xenium_V1_0021981
Saved GeoJSON for Xenium_V1_0021981 to /Volumes/data/UMich/spatial_transcriptomic/qupath/annotation//Xenium_V1_0021981.geojson
Processing Xenium_V1_0021980
Saved GeoJSON for Xenium_V1_0021980 to /Volumes/data/UMich/spatial_transcriptomic/qupath/annotation//Xenium_V1_0021980.geojson
